# Random-mutation hill-climbing (RMHC)

In [18]:
# Librerias
import random
import numpy as np

In [19]:
# Random-mutation hill-climbing
def RMHC(initial_state, cost_funct, new_state_funct, num_iterations = 100, max_or_min = 'max'):
    """
    Random-Mutation Hill Climbing algorithm.

    Parameters:
    - initial_state: Estado inicial del sistema.
    - cost_funct: Función que evalúa el costo (o la calidad) del estado.
    - new_state_funct: Función que genera un nuevo estado a partir del estado actual.
    - num_iterations: Número máximo de iteraciones a ejecutar.
    - stagnation_limit: Número máximo de iteraciones sin mejora antes de detenerse (opcional).

    Returns:
    - current_state: El mejor estado encontrado después de las iteraciones.
    """
    current_state = initial_state
    current_eval = cost_funct(current_state)
    count = 0
    while count < num_iterations:
        new_state = new_state_funct(current_state)
        new_eval = cost_funct(new_state)
        if max_or_min == 'max':
            if new_eval > current_eval:
                current_state = new_state
                current_eval = new_eval
        elif max_or_min == 'min':
            if new_eval < current_eval:
                current_state = new_state
                current_eval = new_eval
        else:
            print('Operación no definida')
        count += 1
    return current_state

### Función cuadrática

$$f(X) = \sum _{i=1} ^{D} x_i ^2 \text{ ,  con  } -10 <= x_i <= 10$$

In [20]:
D = 5 # Tamño del vector de entrada
initial_state_QF = np.array([round(random.choice([-1, 1]) * random.random() * 10, 3) for _ in range(D)]) # Se genera un vector aleatorio con valores en el rango [-10, 10]

In [21]:
def f(state):
    return sum(state**2) # retorna la suma de los cuadrados de los elementos del estado ingresado
    
def random_new_state(current_state):
    indx = random.randint(0, D - 1)
    new_state = current_state.copy()
    new_state[indx] = round(random.choice([-1, 1]) * random.random() * 10, 3) #genera un número aleatorio en el rango de -10 a 10
    return new_state

In [22]:
print(f'Estado inicial: {initial_state_QF}')
print(f'Valor del estado incial: {f(initial_state_QF)}')

Estado inicial: [-0.86   0.097  2.464  2.682 -5.472]
Valor del estado incial: 43.956213000000005


In [23]:
QF_result = RMHC(initial_state_QF, f, random_new_state, 10000, 'min')
print(f'Estado final: {QF_result}')
print(f'Valor del estado final: {f(QF_result)}')

Estado final: [ 0.     0.001  0.003 -0.002 -0.004]
Valor del estado final: 3e-05


### Travel Salesman Problem (TSP)

El objetivo es minimizar la distancia total recorrida. De manera que se puede definir la función objetivo como:

$$
f(\pi) = d(\pi_n, \pi_1) + \sum_{i=1}^{n-1} d(\pi_i, \pi_{i+1}),
$$

donde $d(i, j)$ es la distancia entre la ciudad $i$ y la ciudad $j$. Así, el algoritmo acepta la mutación si $f(\pi_{nuevo}) < f(\pi _{actual})$.

In [24]:
n = 5 # Número de ciudades
# Inicialización del estado (tour inicial aleatorio)
initial_state_TSP = random.sample(range(n), n)  # Generamos una permutación aleatoria de ciudades

In [25]:
distance_matrix = np.zeros([n, n])
for i in range(n):
    for j in range(i):
        distance_matrix[i, j] = distance_matrix[j, i] = random.randint(1, 10)

In [26]:
def calculate_total_distance(tour, dist_matrix = distance_matrix):
    total_dist = 0
    for i in range(len(tour)-1):
        total_dist += dist_matrix[tour[i]][tour[i + 1]]
    total_dist += dist_matrix[tour[-1]][tour[0]]
    return total_dist
    
def generate_random_instance(tour):
    new_tour = tour.copy()
    indx1, indx2 = random.sample(range(len(tour)), 2) # Seleciona dos indices aleatoriamente
    new_tour[indx1], new_tour[indx2] = new_tour[indx2], new_tour[indx1] # Se intercambia el valor de los items con los indicies selecionados anteriormente
    return new_tour

In [27]:
print(f'Estado inicial: {initial_state_TSP}')
print(f'Distancia entre caminos: {distance_matrix}')
print(f'Valor del estado inicial: {calculate_total_distance(initial_state_TSP)}')

Estado inicial: [2, 0, 3, 4, 1]
Distancia entre caminos: [[ 0.  9.  9.  3.  8.]
 [ 9.  0.  2. 10.  7.]
 [ 9.  2.  0.  8.  6.]
 [ 3. 10.  8.  0.  9.]
 [ 8.  7.  6.  9.  0.]]
Valor del estado inicial: 30.0


In [28]:
TSP_result = RMHC(initial_state_TSP, calculate_total_distance, generate_random_instance, 100, 'min')
print(f'Estado final: {TSP_result}')
print(f'Valor del estado final: {calculate_total_distance(TSP_result)}')

Estado final: [4, 0, 3, 2, 1]
Valor del estado final: 28.0


### Knapsack problem

Función heurística:
El objetivo es maximizar el valor total de los artículos seleccionados, sujeto a la restricción de peso.

$$
h(x) = \left\{\begin{matrix}
 -1,\;\;\; \text{if} \;\;\; \sum (w_i \cdot x_i) > \text{Peso máximo}\\
  \sum (v_i \cdot x_i), \;\;\; \text{en caso contrario}
\end{matrix}\right.
$$

In [29]:
n_items = 5 # número de items para elegir
max_weight = 25 # peso máximo permitido
# Define los pesos y valores de los objetos
value_list = np.array([random.randint(1, 50) for _ in range(n_items)])
weight_list = np.array([random.randint(1, 20) for _ in range(n_items)])

In [30]:
#Define el estado inicial aletoriamente
initial_state_KP = np.array([random.randint(0, 1) for _ in range(n_items)])

In [31]:
# Define la función heurística
def h(state, values = value_list, weights = weight_list, maxweight = max_weight):
    value = 0 
    if state @ weights > maxweight:
        value = -1 # Si el valor total de los objetos se pasa del máximo se retorna infinito
    else:
        value = state @ values # En caso contrario se retorna el valor total de los objetos
    return value

In [32]:
# Generar vecinos
def neighbors(state):
    newstate = state.copy()
    idx_to_mutate = random.randint(0, n_items - 1)
    newstate[idx_to_mutate] = 1 - newstate[idx_to_mutate]  # Cambia el valor (si 0 -> 1 y si 1 -> 0)
    return newstate

In [33]:
print(f'Estado inicial: {initial_state_KP}')
print(f'Valores de los items: {value_list}')
print(f'Pesos de los items: {weight_list}')
print(f'Peso máximo permitido: {max_weight}')
print(f'Valor del estado incial: {h(initial_state_KP)}')

Estado inicial: [0 1 0 1 1]
Valores de los items: [29 27 27 20 26]
Pesos de los items: [ 6  9  1  4 19]
Peso máximo permitido: 25
Valor del estado incial: -1


In [34]:
KP_result = RMHC(initial_state_KP, h, neighbors)
print(f'Estado final: {KP_result}')
print(f'Valor del estado final: {h(KP_result)}')
print(f'Peso total de los objetos del estado final: {KP_result @ weight_list}')

Estado final: [0 0 1 1 1]
Valor del estado final: 73
Peso total de los objetos del estado final: 24
